In [1]:
setwd('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/1_Compartment_Analysis/Expt1.3_Cluster Distribution Analyses/Input')
Output = ('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/1_Compartment_Analysis/Expt1.3_Cluster Distribution Analyses/Output')
curdate = "011621"

library(readxl)
library(reshape2)
library(tidyverse)


#reading in cluster data
male_NLF_df <- read_csv("eigencytokines_NLF_male.csv")
male_ELF_df <- read_csv("eigencytokines_ELF_male.csv")
male_Sputum_df <- read_csv("eigencytokines_Sputum_male.csv")
male_Serum_df <- read_csv("eigencytokines_Serum_male.csv")

female_NLF_df <- read_csv("eigencytokines_NLF_female.csv")
female_ELF_df <- read_csv("eigencytokines_ELF_female.csv")
female_Sputum_df <- read_csv("eigencytokines_Sputum_female.csv")
female_Serum_df <- read_csv("eigencytokines_Serum_female.csv")

#reading in demographics file
demographics_data_df = data.frame(read_excel("SubjectInfo_102920.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  X1 = col_character(),
  Cluster1 = col_double(),
  Cluster2 = col_double(),
  Cluster3 = col_double()
)

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  X1 = col_character(),
  Cluster1 = col_double(),
  Cluster2 = col_double(),
  Cluster3 = col_double()
)

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  X1 = col_character(),
  Cluster1 = col_double(),
  Cluster2 = col_double(),
  Cluster3 = col_doubl

In [2]:
#specifying colnames
set_colnames = function(df){
    colnames(df)  = c('SubjectID', 'Cluster1', 'Cluster2', 'Cluster3')
    return(df)
}

#calling fn
male_NLF_df = set_colnames(male_NLF_df)
male_ELF_df = set_colnames(male_ELF_df)
male_Sputum_df = set_colnames(male_Sputum_df)
male_Serum_df = set_colnames(male_Serum_df)

female_NLF_df = set_colnames(female_NLF_df)
female_ELF_df = set_colnames(female_ELF_df)
female_Sputum_df = set_colnames(female_Sputum_df)
female_Serum_df = set_colnames(female_Serum_df)

In [3]:
add_compartment = function(dataframe, compartment_name){
    #"""
    #Takes the dataframes already made from above and adds a column specifying the compartment.

    #:param: compartment dataframe and compartment name
    #:output: a 24x5 matrix containing clustered concentrations. 
    #"""
    Compartment = rep(compartment_name, times = length(dataframe$SubjectID))
    combined_df = cbind(dataframe,Compartment)
    return(combined_df)
}

#calling fn
male_NLF_df = add_compartment(male_NLF_df,'NLF')
male_ELF_df = add_compartment(male_ELF_df, 'ELF')
male_Sputum_df = add_compartment(male_Sputum_df, 'Sputum')
male_Serum_df = add_compartment(male_Serum_df, 'Serum')

female_NLF_df = add_compartment(female_NLF_df,'NLF')
female_ELF_df = add_compartment(female_ELF_df, 'ELF')
female_Sputum_df = add_compartment(female_Sputum_df, 'Sputum')
female_Serum_df = add_compartment(female_Serum_df, 'Serum')

#making one df with all these values 
male_initial_clustered_data_df = rbind(male_NLF_df, male_ELF_df, male_Sputum_df, male_Serum_df)
male_clustered_data_df = melt(male_initial_clustered_data_df, variable = 'Cluster', value.name = 'Conc_pslog2')

female_initial_clustered_data_df = rbind(female_NLF_df, female_ELF_df, female_Sputum_df, female_Serum_df)
female_clustered_data_df = melt(female_initial_clustered_data_df, variable = 'Cluster', value.name = 'Conc_pslog2')

#combining data_df and subject info
male_data_df = inner_join(male_clustered_data_df, demographics_data_df) %>%
    select(!c(Serum_Cotinine, Ethnicity, Age, BMI))
female_data_df = inner_join(female_clustered_data_df, demographics_data_df) %>%
    select(!c(Serum_Cotinine, Ethnicity, Age, BMI))
head(male_data_df)

Using SubjectID, Compartment as id variables

Using SubjectID, Compartment as id variables

Joining, by = "SubjectID"

Joining, by = "SubjectID"



,SubjectID,Compartment,Cluster,Conc_pslog2,SubjectNo,Group,Race,Sex
,<chr>,<chr>,<fct>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,CS_23,NLF,Cluster1,-0.29227498,23,CS,W,M
2,CS_24,NLF,Cluster1,-0.12942120,24,CS,W,M
3,CS_25,NLF,Cluster1,-0.08715958,25,CS,W,M
4,CS_26,NLF,Cluster1,-0.33795856,26,CS,AA,M
5,CS_27,NLF,Cluster1,-0.28029875,27,CS,W,M
6,Ecig_33,NLF,Cluster1,-0.33290255,33,Ecig,As,M


Since individual cytokines showed the most differences in male e-cig users and female cigarette smokers, we'll test the same differences in their clustered cytokines. 

In [4]:
#creating dfs for each stratified demographic variable
female_NS = female_data_df %>%
    filter(Group == 'NS')
female_CS = female_data_df %>%
    filter(Group == 'CS')
male_NS = male_data_df %>%
    filter(Group == 'NS')
male_Ecig = male_data_df %>%
    filter(Group == 'Ecig')


#creating vectors to loop through
compartment = c('NLF','ELF','Sputum','Serum')
cluster = unique(male_data_df$Cluster)

In [14]:
#initializing vectors to store values
female_CS_vector = c()
male_Ecig_vector = c()
wilcoxon_rank_sum_values = function(df1, df2, empty_vector){
    #
    for (i in 1:length(compartment)){
        for (j in 1: length(cluster)){
            variable1_df = df1 %>% # baseline df
                filter(Compartment == compartment[i], Cluster == cluster[j]) %>%
                select(Compartment, Cluster, Conc_pslog2)
            variable2_df = df2 %>% # smoker df
                filter(Compartment == compartment[i], Cluster == cluster[j]) %>%
                select(Compartment, Cluster, Conc_pslog2)


            #running wilcoxon rank sum
            wilcox_test = wilcox.test(variable1_df$Conc_pslog2, variable2_df$Conc_pslog2)

            #contains compartment, cluster, u stat, p value
            values_vector = c(compartment[i], cluster[j], wilcox_test$statistic, wilcox_test$p.value)
            empty_vector = c(empty_vector, values_vector)

        }
    }
    return(empty_vector)
}

#calling fn
female_CS_wilcoxon_values = wilcoxon_rank_sum_values(female_NS, female_CS, female_CS_vector)
male_Ecig_wilcoxon_values = wilcoxon_rank_sum_values(male_NS, male_Ecig, male_Ecig_vector)

In [31]:
final_table = function(vector){
    #"""
    #Reformatting each vector into matrices to export. 
    
    #:param: vector, demographic variable
    #:output: a 5x12 matrix containing compartment, cluster, protein, u stat, p value, p adj

    #"""
    dim(vector) = c(4, 12)
    table = data.frame(t(vector))
    colnames(table) = c('Compartment', 'Cluster', 'Stat', 'P Value')
    
    #going back to calculate padj values
    PAdj = c()
    for (i in 1:length(compartment)){
        single_compartment_df = table %>%
            filter(Compartment == compartment[i])
        padj =  p.adjust(single_compartment_df[,4], method = "fdr") 
        PAdj = c(PAdj, padj)
        
    }
    
    table = cbind(table, PAdj)
    return(table)
}

#calling fn
male_Ecig_table = final_table(male_Ecig_wilcoxon_values)
male_Ecig_table
female_CS_table = final_table(female_CS_wilcoxon_values)
female_CS_table

Compartment,Cluster,Stat,P Value,PAdj
<chr>,<chr>,<chr>,<chr>,<dbl>
NLF,1,38,0.812525709584533,0.96225833
NLF,2,36,0.962258329905389,0.96225833
NLF,3,62,0.00678733031674208,0.02036199
ELF,1,38,0.812525709584533,0.81252571
ELF,2,32,0.812525709584533,0.81252571
ELF,3,28,0.536199095022624,0.81252571
Sputum,1,49,0.193233237350884,0.57969971
Sputum,2,28,0.536199095022624,0.66906623
Sputum,3,40,0.669066227889757,0.66906623


Compartment,Cluster,Stat,P Value,PAdj
<chr>,<chr>,<chr>,<chr>,<dbl>
NLF,1,31,0.778865578865579,0.9550894
NLF,2,29,0.955089355089355,0.9550894
NLF,3,12,0.0721056721056721,0.2163170
ELF,1,26,0.866511266511267,0.8665113
ELF,2,20,0.396891996891997,0.5953380
ELF,3,36,0.396891996891997,0.5953380
Sputum,1,34,0.535819735819736,0.5358197
Sputum,2,40,0.189277389277389,0.5358197
Sputum,3,22,0.535819735819736,0.5358197
